In [6]:
import pandas as pd
import matplotlib.pyplot as plt

from modules.loader import ImmothepDataLoader
loader = ImmothepDataLoader()
del(ImmothepDataLoader)

from modules.csvLoader import csvLoader

DATASET_PATH = '../datas/RAW/'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Lecture des csv

In [7]:
maison_filtre = csvLoader.load('maison_filtre')

Successfully loaded  maison_filtre.csv


## Observation sur les types de mutation et leur influence sur la valeur de la maison
```
nb vente :	 489535 	soit	 100  % des maisons
nb vente_achevement :	 253 	soit	 0.1  % des maisons
nb echange :	    694 	soit	 0.1  % des maisons
nb adjudication :	 1124 	soit	 0.2  % des maisons
nb vente_terrain :	 132 	soit	 0.0  % des maisons
nb expropriation :	 10 	soit	 0.0  % des maisons
```

In [8]:
vente = maison_filtre[maison_filtre['Nature mutation'] == 'Vente']
vente_achevement = maison_filtre[maison_filtre['Nature mutation'] == "Vente en l'état futur d'achèvement"]
echange = maison_filtre[maison_filtre['Nature mutation'] == "Echange"]
adjudication = maison_filtre[maison_filtre['Nature mutation'] == "Adjudication"]
vente_terrain = maison_filtre[maison_filtre['Nature mutation'] == "Vente terrain à bâtir"]
expropriation = maison_filtre[maison_filtre['Nature mutation'] == "Expropriation"]

print('nb vente :\t', len(vente), '\tsoit\t', round(len(vente)/len(maison_filtre)*100), ' % des maisons' )
print('nb vente_achevement :\t', len(vente_achevement), '\tsoit\t', round(len(vente_achevement)/len(maison_filtre)*100, 1), ' % des maisons' )
print('nb echange :\t', len(echange), '\tsoit\t', round(len(echange)/len(maison_filtre)*100, 1), ' % des maisons' )
print('nb adjudication :\t', len(adjudication), '\tsoit\t', round(len(adjudication)/len(maison_filtre)*100, 1), ' % des maisons' )
print('nb vente_terrain :\t', len(vente_terrain), '\tsoit\t', round(len(vente_terrain)/len(maison_filtre)*100, 1), ' % des maisons' )
print('nb expropriation :\t', len(expropriation), '\tsoit\t', round(len(expropriation)/len(maison_filtre)*100, 1), ' % des maisons' )

print()

del(vente, vente_achevement, echange, adjudication, vente_terrain, expropriation)

nb vente :	 489535 	soit	 100  % des maisons
nb vente_achevement :	 253 	soit	 0.1  % des maisons
nb echange :	 694 	soit	 0.1  % des maisons
nb adjudication :	 1124 	soit	 0.2  % des maisons
nb vente_terrain :	 132 	soit	 0.0  % des maisons
nb expropriation :	 10 	soit	 0.0  % des maisons



In [41]:
maison_valeur = maison_filtre[['Nature mutation', 'Valeur fonciere']]
print(len(maison_valeur))
# print( maison_valeur.groupby(['Nature mutation']).mean() )

echange = maison_valeur['Nature mutation'] == "Echange"
print(echange.sum())

echange = maison_valeur[[maison_valeur["Nature mutation"] =="Echange"]]
print(echange.head())
echange['Valeur fonciere'].mean()

491748
1124


ValueError: Item wrong length 1 instead of 491748.